<a href="https://colab.research.google.com/github/Ani-Michael/Analyzing-timbre-and-predicting-emotional-trajectory-in-Nigerian-music-with-Deep-learning/blob/main/My_model_with_feature_column.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive


Importing data from drive

In [3]:
import pandas as pd
df=pd.read_csv("/content/labeled_dataset.csv")
df.head(10)

,name,album,artist,artist_top_genre,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,emotion
0,Sparky,Mandy & The Jungle,Cruel Santino,alternative r&b,2019,144000,48,0.666,0.8510,0.420,0.534000,0.1100,-6.699,0.0829,133.015,5,Neutral
1,shuga rush,EVERYTHING YOU HEARD IS TRUE,Odunsi (The Engine),afropop,2020,89488,30,0.710,0.0822,0.683,0.000169,0.1010,-5.640,0.3600,129.993,3,Excited
2,LITT!,LITT!,AYLØ,indie r&b,2018,207758,40,0.836,0.2720,0.564,0.000537,0.1100,-7.127,0.0424,130.005,4,Neutral
3,Confident / Feeling Cool,Enjoy Your Life,Lady Donli,nigerian pop,2019,175135,14,0.894,0.7980,0.611,0.000187,0.0964,-4.961,0.1130,111.087,4,Neutral
4,wanted you,rare.,Odunsi (The Engine),afropop,2018,152049,25,0.702,0.1160,0.833,0.910000,0.3480,-6.044,0.0447,105.115,4,Happy
5,Kasala,Pioneers,DRB Lasgidi,nigerian pop,2020,184800,26,0.803,0.1270,0.525,0.000007,0.1290,-10.034,0.1970,100.103,4,Neutral
6,Pull Up,Everything Pretty,prettyboydo,nigerian pop,2018,202648,29,0.818,0.4520,0.587,0.004490,0.5900,-9.840,0.1990,95.842,4,Neutral
7,take a break,rare.,Odunsi (The Engine),afropop,2018,141933,27,0.808,0.6080,0.300,0.000048,0.0863,-11.213,0.0453,119.964,4,Neutral
8,Cash,Enjoy Your Life,Lady Donli,nigerian pop,2019,187714,36,0.846,0.2140,0.669,0.467000,0.0857,-7.822,0.0441,115.008,4,Neutral
9,SATISFIED,GEMINI,Tay Iwar,alternative r&b,2019,123082,30,0.555,0.9120,0.295,0.275000,0.0967,-11.038,0.0360,77.033,4,Neutral


In [4]:
df.describe()

,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
count,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000
mean,2015.390566,222298.169811,17.507547,0.741619,0.265412,0.760623,0.016305,0.147308,-4.953011,0.130748,116.487864,3.986792
std,3.131688,39696.822259,18.992212,0.117522,0.208342,0.148533,0.090321,0.123588,2.464186,0.092939,23.518601,0.333701
min,1998.000000,89488.000000,0.000000,0.255000,0.000665,0.111000,0.000000,0.028300,-19.362000,0.027800,61.695000,3.000000
25%,2014.000000,199305.000000,0.000000,0.681000,0.089525,0.669000,0.000000,0.075650,-6.298750,0.059100,102.961250,4.000000
50%,2016.000000,218509.000000,13.000000,0.761000,0.220500,0.784500,0.000004,0.103500,-4.558500,0.097950,112.714500,4.000000
75%,2017.000000,242098.500000,31.000000,0.829500,0.403000,0.875750,0.000234,0.164000,-3.331000,0.177000,125.039250,4.000000
max,2020.000000,511738.000000,73.000000,0.966000,0.954000,0.995000,0.910000,0.811000,0.582000,0.514000,206.007000,5.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              530 non-null    object 
 1   album             530 non-null    object 
 2   artist            530 non-null    object 
 3   artist_top_genre  530 non-null    object 
 4   release_date      530 non-null    int64  
 5   length            530 non-null    int64  
 6   popularity        530 non-null    int64  
 7   danceability      530 non-null    float64
 8   acousticness      530 non-null    float64
 9   energy            530 non-null    float64
 10  instrumentalness  530 non-null    float64
 11  liveness          530 non-null    float64
 12  loudness          530 non-null    float64
 13  speechiness       530 non-null    float64
 14  tempo             530 non-null    float64
 15  time_signature    530 non-null    int64  
 16  emotion           530 non-null    object 
dt

Data processing

In [6]:
# Drop non-numeric columns that aren't useful for modeling
cols_to_drop = ['name', 'album', 'artist', 'artist_top_genre', 'release_date', 'length', 'popularity']
df_new_data = df.drop(columns=[col for col in cols_to_drop if col in df.columns], errors='ignore')

In [7]:
df_new_data.head(10)

,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,emotion
0,0.666,0.8510,0.420,0.534000,0.1100,-6.699,0.0829,133.015,5,Neutral
1,0.710,0.0822,0.683,0.000169,0.1010,-5.640,0.3600,129.993,3,Excited
2,0.836,0.2720,0.564,0.000537,0.1100,-7.127,0.0424,130.005,4,Neutral
3,0.894,0.7980,0.611,0.000187,0.0964,-4.961,0.1130,111.087,4,Neutral
4,0.702,0.1160,0.833,0.910000,0.3480,-6.044,0.0447,105.115,4,Happy
5,0.803,0.1270,0.525,0.000007,0.1290,-10.034,0.1970,100.103,4,Neutral
6,0.818,0.4520,0.587,0.004490,0.5900,-9.840,0.1990,95.842,4,Neutral
7,0.808,0.6080,0.300,0.000048,0.0863,-11.213,0.0453,119.964,4,Neutral
8,0.846,0.2140,0.669,0.467000,0.0857,-7.822,0.0441,115.008,4,Neutral
9,0.555,0.9120,0.295,0.275000,0.0967,-11.038,0.0360,77.033,4,Neutral


In [8]:
# dropping missing values
df_new_data.dropna()

,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,emotion
0,0.666,0.8510,0.420,0.534000,0.1100,-6.699,0.0829,133.015,5,Neutral
1,0.710,0.0822,0.683,0.000169,0.1010,-5.640,0.3600,129.993,3,Excited
2,0.836,0.2720,0.564,0.000537,0.1100,-7.127,0.0424,130.005,4,Neutral
3,0.894,0.7980,0.611,0.000187,0.0964,-4.961,0.1130,111.087,4,Neutral
4,0.702,0.1160,0.833,0.910000,0.3480,-6.044,0.0447,105.115,4,Happy
...,...,...,...,...,...,...,...,...,...,...
525,0.735,0.6320,0.918,0.193000,0.0718,-2.582,0.0355,127.954,4,Happy
526,0.602,0.1770,0.883,0.000004,0.1190,-6.183,0.0778,154.010,4,Happy
527,0.729,0.2380,0.864,0.000000,0.1950,-3.942,0.0668,121.058,4,Happy
528,0.809,0.0490,0.806,0.000000,0.0659,-6.547,0.2340,123.997,4,Happy


In [9]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
# Encode Emotion Labels
label_encoder = LabelEncoder()
df_new_data['emotion'] = label_encoder.fit_transform(df['emotion'])

In [10]:
df_new_data.head(10)

,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,emotion
0,0.666,0.8510,0.420,0.534000,0.1100,-6.699,0.0829,133.015,5,3
1,0.710,0.0822,0.683,0.000169,0.1010,-5.640,0.3600,129.993,3,1
2,0.836,0.2720,0.564,0.000537,0.1100,-7.127,0.0424,130.005,4,3
3,0.894,0.7980,0.611,0.000187,0.0964,-4.961,0.1130,111.087,4,3
4,0.702,0.1160,0.833,0.910000,0.3480,-6.044,0.0447,105.115,4,2
5,0.803,0.1270,0.525,0.000007,0.1290,-10.034,0.1970,100.103,4,3
6,0.818,0.4520,0.587,0.004490,0.5900,-9.840,0.1990,95.842,4,3
7,0.808,0.6080,0.300,0.000048,0.0863,-11.213,0.0453,119.964,4,3
8,0.846,0.2140,0.669,0.467000,0.0857,-7.822,0.0441,115.008,4,3
9,0.555,0.9120,0.295,0.275000,0.0967,-11.038,0.0360,77.033,4,3


In [11]:
# Select Features & Target
X = df_new_data.drop(columns=['emotion'])
y = df_new_data['emotion']

In [12]:
# Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-0.64405173,  2.81336503, -2.29540567, ..., -0.51532247,
         0.70339004,  3.03914122],
       [-0.26930034, -0.88021579, -0.52308752, ...,  2.46902286,
         0.57477461, -2.95990849],
       [ 0.80385138,  0.03164899, -1.32501094, ..., -0.9515043 ,
         0.57528532,  0.03961637],
       ...,
       [-0.10747587, -0.13169876,  0.69664475, ..., -0.68871821,
         0.19450364,  0.03961637],
       [ 0.5738903 , -1.03972006,  0.30579132, ...,  1.11201271,
         0.31958661,  0.03961637],
       [ 0.63350984, -0.7274376 ,  1.35031342, ...,  0.08887014,
        -0.40499238,  0.03961637]])

In [13]:
from sklearn.model_selection import train_test_split
# Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape for LSTM (Adding a time step dimension)
time_steps = 1  # Since we treat each song as a single sequence
X_train_reshaped = X_train.reshape((X_train.shape[0], time_steps, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], time_steps, X_test.shape[1]))

Model training

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, Flatten, Reshape
# Define Deep Learning Model (Hybrid CNN + LSTM)
model = Sequential([
    Reshape((time_steps, X_train.shape[1]), input_shape=(time_steps, X_train.shape[1])),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')  # Number of emotion categories
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# Compile Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(X_train_reshaped, y_train, epochs=30, batch_size=16, validation_data=(X_test_reshaped, y_test))

# Evaluate Model
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.4867 - loss: 1.5843 - val_accuracy: 0.5755 - val_loss: 1.5017
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6623 - loss: 1.4602 - val_accuracy: 0.6604 - val_loss: 1.3164
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6483 - loss: 1.2294 - val_accuracy: 0.6698 - val_loss: 1.0036
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6769 - loss: 0.9035 - val_accuracy: 0.7170 - val_loss: 0.7529
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7381 - loss: 0.6883 - val_accuracy: 0.8491 - val_loss: 0.5729
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8099 - loss: 0.5767 - val_accuracy: 0.8868 - val_loss: 0.4347
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8359 - loss: 0.4750 - val_accuracy: 0.8868 - val_loss: 0.3611
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8544 - loss: 0.4281 - val_accuracy: 0.9340 - val_loss

In [16]:
import joblib

# Save the StandardScaler
joblib.dump(scaler, "scaler.pkl")

# Save the LabelEncoder
joblib.dump(label_encoder, "label_encoder.pkl")

# Save the order and names of features used in training
feature_columns = X.columns.tolist()
joblib.dump(feature_columns, "feature_columns.pkl")

['feature_columns.pkl']

In [17]:
model.save("Final_Final_test.keras")  # Saves the entire model (architecture + weights)

Testing Model

In [18]:
from tensorflow import keras
Model = keras.models.load_model("/content/Final_Final_test.keras")

In [20]:
new_data = pd.read_csv("/content/nigerian_spotify_songs1 (1).csv")

In [21]:
# Drop non-numeric columns that aren't useful for modeling
cols_to_drop = ['name', 'album', 'artist', 'artist_top_genre', 'release_date', 'length', 'popularity']
new_data_drop = new_data.drop(columns=[col for col in cols_to_drop if col in new_data.columns], errors='ignore')

In [22]:
new_data_drop.head(10)

,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,0.666,0.8510,0.420,0.534000,0.1100,-6.699,0.0829,133.015,5
1,0.710,0.0822,0.683,0.000169,0.1010,-5.640,0.3600,129.993,3
2,0.836,0.2720,0.564,0.000537,0.1100,-7.127,0.0424,130.005,4
3,0.894,0.7980,0.611,0.000187,0.0964,-4.961,0.1130,111.087,4
4,0.702,0.1160,0.833,0.910000,0.3480,-6.044,0.0447,105.115,4
5,0.803,0.1270,0.525,0.000007,0.1290,-10.034,0.1970,100.103,4
6,0.818,0.4520,0.587,0.004490,0.5900,-9.840,0.1990,95.842,4
7,0.808,0.6080,0.300,0.000048,0.0863,-11.213,0.0453,119.964,4
8,0.846,0.2140,0.669,0.467000,0.0857,-7.822,0.0441,115.008,4
9,0.555,0.9120,0.295,0.275000,0.0967,-11.038,0.0360,77.033,4


In [23]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = joblib.load("/content/scaler.pkl")  # Load the scaler used during training
new_data_scaled = scaler.transform(new_data_drop)

# Reshape new data to match model input (samples, time steps, features)
new_data_reshaped = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

# Make predictions
predictions = model.predict(new_data_reshaped)
print(predictions)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
[[6.5887211e-06 1.0443386e-02 6.9168731e-05 9.8947632e-01 4.6248770e-06]
 [6.0064781e-02 5.7863533e-01 3.1533163e-02 2.8546372e-01 4.4303101e-02]
 [9.1611629e-04 1.9054854e-01 1.6757277e-03 8.0498701e-01 1.8725996e-03]
 ...
 [5.5464803e-05 1.6061812e-06 9.9990541e-01 3.7453632e-05 1.6838333e-08]
 [1.5073537e-04 1.0704862e-05 9.9972647e-01 1.1189093e-04 1.5271908e-07]
 [1.1425923e-06 1.3637256e-08 9.9999797e-01 8.6132633e-07 2.0882776e-11]]


In [24]:
# Get predicted class indices (0 to 4)
predicted_classes = predictions.argmax(axis=1)


In [25]:
# Decode back to emotion labels
predicted_emotions = label_encoder.inverse_transform(predicted_classes)

In [26]:
#combine with original data (optional)
output_df = pd.read_csv("/content/nigerian_spotify_songs1 (1).csv")
output_df['Predicted Emotion'] = predicted_emotions

In [27]:
#show the results
print(output_df[['name', 'Predicted Emotion']] if 'name' in output_df.columns else output_df[['Predicted Emotion']])


                         name Predicted Emotion
0                      Sparky           Neutral
1                  shuga rush           Excited
2                       LITT!           Neutral
3    Confident / Feeling Cool           Neutral
4                  wanted you             Happy
..                        ...               ...
525                   Kwaroro             Happy
526             Concert Party             Happy
527                      Loko             Happy
528                      Gift             Happy
529                    Macoma             Happy

[530 rows x 2 columns]


In [28]:
# Create a DataFrame for predicted emotions
import pandas as pd  # Make sure pandas is imported
print_df = pd.DataFrame({'predicted_emotions': predicted_emotions})

In [29]:
print_df.to_csv("emotion_predictions_mixed22.csv", index=False)
